# Part I. ETL Pipeline for Pre-Processing the Files

#### Importing Python packages 

In [ ]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
                
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
# print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Completing the Apache Cassandra coding portion of the project. 

#### Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Creating Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Setting Keyspace

In [ ]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Creating queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1

In [ ]:
query = """CREATE TABLE IF NOT EXISTS music_query1 
           (artist text, song text, length float, sessionId int, itemInSession int, 
            PRIMARY KEY(sessionId, itemInSession))"""
# sessionId and itemInSession are in the WHERE clause, meaning we're filtering by them, 
# that's why I choose them for PK.
session.execute(query)
print("Table created")

Table created


In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO music_query1 (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        
print("Data inserted")

Data inserted


In [ ]:
query = "SELECT artist, song, length FROM music_query1 WHERE sessionId = 338 AND itemInSession = 4"
rows = session.execute(query)

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2

In [ ]:
query1 = """CREATE TABLE IF NOT EXISTS music_query2 
        (artist text, song text, ufirst text, ulast text, userId int, itemInSession int, sessionId int, 
        PRIMARY KEY ((userId, sessionId), itemInSession))"""
# userId and sessionId are in the WHERE clause, meaning we're filtering by them, 
# that's why I choose them for PK, and we're ordering by itemInSession,
# so it should be a clustering column.
session.execute(query1)
print("Table created")

Table created


In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO music_query2 (artist, song, ufirst, ulast, userId, itemInSession, sessionId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[3]), int(line[8])))
        
print("Data inserted")

Data inserted


In [17]:
query = """SELECT artist, song, ufirst, ulast
           FROM music_query2
           WHERE userId = 10 AND sessionId = 182
           ORDER BY itemInSession"""

rows = session.execute(query)

for row in rows: 
    print(row.artist, row.song, row.ufirst, row.ulast)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3

In [ ]:
query = """CREATE TABLE IF NOT EXISTS music_query3 
        (ufirst text, ulast text, song text, userId int,
        PRIMARY KEY (song, userId))"""
# song is in the WHERE clause, meaning we're filtering by it, 
# that's why I choose it for PK, and we also need the names to be unique,
# so userId is set as a clustering column.
session.execute(query)
print("Table created")

Table created


In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO music_query3 (ufirst, ulast, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))
        
print("Data inserted")

Data inserted


In [30]:
query = """SELECT ufirst, ulast
            FROM music_query3
            WHERE song = 'All Hands Against His Own'"""
rows = session.execute(query)

for row in rows:
    print(row.ufirst, row.ulast)

Jacqueline Lynch
Tegan Levine
Sara Johnson


# Dropping the tables and closing the session

In [24]:
query = "DROP TABLE IF EXISTS music_query1"
session.execute(query)

In [25]:
query = "DROP TABLE IF EXISTS music_query2"
session.execute(query)

In [31]:
query = "DROP TABLE IF EXISTS music_query3"
session.execute(query)

In [32]:
session.shutdown()
cluster.shutdown()